# Compute cost matrix on a small graph by replicating data

cuGraph currently does not have a All-Source Shortest Path (ASSP) algorithm.  One is on the roadmap, but that doesn't help us today.  If the graph to be processed is small, then it is possible to run ASSP by creating a lot of copies of the graph and running one seed per graph copy.


| Author Credit |    Date    |  Update          | cuGraph Version |  Test Hardware |
| --------------|------------|------------------|-----------------|----------------|
| Brad Rees     | 06/21/2022 | created          | 22.08           | V100 w 32 GB, CUDA 11.5
| Don Acosta    | 06/28/2022 | modified         | 22.08           | V100 w 32 GB, CUDA 11.5

In [10]:
# system and other
import gc
import os
import time
from time import perf_counter
import math

# rapids
import cugraph
import cudf

-----
# The first section discuss each step independently.  The second section puts it all together

Let's start with data read

In [11]:
# simple function to read in the CSV data file
def read_data_cudf(datafile):
    gdf = cudf.read_csv(datafile,
                     delimiter=" ",
                     header=None,
                     names=['src','dst', 'wt'])
    return gdf

In [12]:
# function to determine the number of nodes in the dataset
def find_number_of_nodes(df):
    node = cudf.concat([df['src'], df['dst']])
    node = node.unique()
    return len(node)

### Read the data and verify that it is zero based (e.g. first vertex is 0)

In [13]:
t1 = perf_counter()
gdf = read_data_cudf('../data/email-Eu-core.csv')
read_t = perf_counter() - t1

In [14]:
print(f" read {len(gdf)} edges in {read_t} seconds")

 read 25571 edges in 0.005591400200501084 seconds


In [15]:
# verify that the starting ID is zero
min([gdf['src'].min(), gdf['dst'].min()])

0

In [16]:
# check the max ID
max([gdf['src'].max(), gdf['dst'].max()])

1004

In [17]:
# the number of nodes should be one greater than the max ID
# that is the ID that we start the next instance of the data at
offset = find_number_of_nodes(gdf)
print(offset)

1005


## Now let's dive into how to replicate the data
We will use a model that doubles the data at each pass.  That is a lot faster 
than adding one copy at a time.  
The number of disjoint versions of the data will be a power of 2.
Although the power of 2 replication results in faster data set growth growth and Graph building, the simple order one replication is shown here for illustration purposes.


![Data Duplicated](../../notebooks/img/graph_after_replication.png)

In [18]:
# This function creates additional version of the data 

def make_data(base_df, N):
    id = find_number_of_nodes(base_df)
    _d = base_df

    for x in range(N):
        tmp = _d.copy()
        tmp['src'] += id
        tmp['dst'] += id
        _d = cudf.concat([_d,tmp])
        id = id * 2
    return _d

In [19]:
%%timeit
_ = make_data(gdf, 3)

11.1 ms ± 82.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
%%time
gdf2 = make_data(gdf, 3)
print()


CPU times: user 17 ms, sys: 470 µs, total: 17.5 ms
Wall time: 16.1 ms


In [21]:
# simple print to show tha there is not a lot more data
# print # of Edges and # of Nodes
print(f"Old {len(gdf)} {find_number_of_nodes(gdf)}")
print(f"New {len(gdf2)} {find_number_of_nodes(gdf2)}")

Old 25571 1005
New 204568 8040


## Build the ghost node connection set
A ghost node is an artificially added node to parallelize/simulate the all-points shortest path algorithm which is not yet supported.
After the ghost node is added, the 2nd hop is actually the all points shortest path.
The Ghost node is later removed after the Shortest path algorithms are run.

![Ghost Node](../../notebooks/img/graph_after_ghost.png)

The Ghost Node is connected to a different corresponding node in each replication so all sources are covered.

In this simple example of a four-node 'square' graph after complete replication and adding the ghost node, the graph looks like this:

![Ghost Node](../../notebooks/img/Full-four_node_replication.png)





In [22]:
def add_ghost_node(_df, N):
    # get the size of the graph.  That number will be the ghost node ID
    ghost_node_id = find_number_of_nodes(_df)
    
    num_copies = math.floor(math.pow(2, N))

    seeds = cudf.DataFrame()
    seeds['dst'] = [((offset * x) + x) for x in range(num_copies)]
    seeds['src'] = ghost_node_id
    
    _d = cudf.concat([_df, seeds])
    
    return _d, ghost_node_id

In [23]:
%%timeit
_, _ = add_ghost_node(gdf2, 10)

8.23 ms ± 265 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
gdf_with_ghost, ghost_id = add_ghost_node(gdf2, 10)

## Create an Empty directed Graph

In [25]:
G = cugraph.Graph(directed=True)

Populate the new graph with an edgelist containing
* The original Data
* The replicated data copies
* Each replication connected to the Ghost Node by a single edge from a different node
in each copy of the graph.

In [26]:
%time
G.from_cudf_edgelist(gdf_with_ghost, source='src', destination='dst', renumber=False)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.44 µs


In [27]:
%time
G.number_of_edges()

CPU times: user 1e+03 ns, sys: 1e+03 ns, total: 2 µs
Wall time: 5.48 µs


205592

### Run SSSP from the ghost node
The single Ghost node source becomes a all-source shortest path after one hop since all the
replicated data is connected through that node.

In [28]:
%%timeit
X = cugraph.sssp(G, ghost_id)

20.2 ms ± 157 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [29]:
X = cugraph.sssp(G, ghost_id)

This result will contain a ghost node like the simple example.

In [30]:
X.head(5)

,distance,vertex,predecessor
0,1.0,0,8040
1,2.0,1,0
2,3.0,2,5
3,3.0,3,6
4,3.0,4,6


## Now reset vertex IDs and convert to a cost matrix
The Cost Matrix

In [31]:
# drop the ghost node which doesnt exist so remove from matrix.
X = X[X['predecessor'] != ghost_id]

In [32]:
# drop unreachable
X = cugraph.filter_unreachable(X)

In [33]:
# adjust distances so that they don't go to the ghost node
X['distance'] -= 1

## Now the Ghost node and tangential edges are removed.

In [34]:
X.head(5)

,distance,vertex,predecessor
1,1.0,1,0
2,2.0,2,5
3,2.0,3,6
4,2.0,4,6
5,1.0,5,0


In [35]:
# add a new column for the seed
# since each seed was a different component with a different offset amount, exploit that to determine the seed number
X['seed'] = (X['vertex'] / offset).astype(int)

In [36]:
X.head(5)

,distance,vertex,predecessor,seed
1,1.0,1,0,0
2,2.0,2,5,0
3,2.0,3,6,0
4,2.0,4,6,0
5,1.0,5,0,0


In [37]:
# Now adjust all vertices to be in the correct range
# resets the seed number to the
X['v2'] = X['vertex'] - (X['seed'] * offset)

In [38]:
# Finally just pull out the cost matrix
cost = X.drop(columns=['vertex', 'predecessor'])

In [39]:
cost.head(10)

,distance,seed,v2
1,1.0,0,1
2,2.0,0,2
3,2.0,0,3
4,2.0,0,4
5,1.0,0,5
6,1.0,0,6
7,2.0,0,7
8,2.0,0,8
9,2.0,0,9
10,3.0,0,10


In [40]:
# cleanup 
del G
del X
del gdf_with_ghost
del gdf2


## Now do it all in a single function

In [41]:
N = 10

In [42]:
def build_cost_matrix(_gdf):
    data = make_data(_gdf, N)
    gdf_with_ghost, ghost_id = add_ghost_node(data, N)
    G = cugraph.Graph(directed=True)
    G.from_cudf_edgelist(gdf_with_ghost, source='src', destination='dst', renumber=False)
    X = cugraph.sssp(G, ghost_id)
    X = X[X['predecessor'] != ghost_id]
    X = cugraph.filter_unreachable(X)
    X['distance'] -= 1
    X['seed'] = (X['vertex'] / offset).astype(int)
    X['v2'] = X['vertex'] - (X['seed'] * offset)
    cost = X.drop(columns=['vertex', 'predecessor'])
    return X

In [43]:
%%timeit
CM = build_cost_matrix(gdf)
CM

404 ms ± 2.71 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
CM = build_cost_matrix(gdf)
CM.head(5)

,distance,vertex,predecessor,seed,v2
1,1.0,1,0,0,1
2,2.0,2,5,0,2
3,2.0,3,6,0,3
4,2.0,4,6,0,4
5,1.0,5,0,0,5


___
Copyright (c) 2022, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___